<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [77]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [78]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "C4SS8dsi1TSlBcSM.FQPsjTGnxcG3aKFbWq1yzAaBw0jRlpCtRB2l7ESP9MGqHKgjMCRbpjOjbdPzJwLp"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163417
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "Sweden", "city": "Majorna-Linn\\u00e9", "street": "Masthugget", "aqicn_url": "https://api.waqi.info/feed/A60853/", "latitude": 57.7, "longitude": 11.94}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [79]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [80]:
import requests
import pandas as pd


aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

hellooo
<Response [200]> response
200 response.status_code
{'status': 'ok', 'data': {'aqi': 23, 'idx': -60853, 'attributions': [{'url': 'https://sensor.community/', 'name': 'Citizen Science project sensor.community', 'station': '13439'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [57.698, 11.946], 'name': 'Tredje Långgatan', 'url': 'https://aqicn.org/station/@60853', 'location': 'Göteborgs Mini-Hotel, 31, Tredje Långgatan, Masthugget, Majorna-Linné, Gothenburg, Göteborg, Västra Götaland County, Region Götaland, 41327, Sweden'}, 'dominentpol': 'pm25', 'iaqi': {'pm10': {'v': 11}, 'pm25': {'v': 23}}, 'time': {'s': '2024-11-13 11:01:23', 'tz': '+01:00', 'v': 1731492083, 'iso': '2024-11-13T10:01:23Z'}}} Data retrieved successfully
{'aqi': 23, 'idx': -60853, 'attributions': [{'url': 'https://sensor.community/', 'name': 'Citizen Science project sensor.community', 'station': '13439'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index

,pm25,country,city,street,date,url
0,23.0,Sweden,Majorna-Linné,Masthugget,2024-11-13,https://api.waqi.info/feed/A60853/


In [81]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [82]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 57.75°N 12.0°E
Elevation 2.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,8.25,0.0,19.523155,202.782440,Majorna-Linné
1,2024-11-14,6.85,0.0,3.096837,324.462250,Majorna-Linné
2,2024-11-15,10.65,0.0,10.685391,237.380768,Majorna-Linné
3,2024-11-16,10.00,0.3,24.400459,248.355499,Majorna-Linné
4,2024-11-17,7.90,0.3,31.018547,248.198532,Majorna-Linné
5,2024-11-18,7.75,0.0,21.031521,276.881622,Majorna-Linné
6,2024-11-19,6.75,0.0,13.979872,235.491409,Majorna-Linné
7,2024-11-20,5.35,0.9,23.770906,91.735672,Majorna-Linné
8,2024-11-21,6.90,0.0,19.523155,174.710007,Majorna-Linné
9,2024-11-22,5.65,0.2,14.587776,254.248749,Majorna-Linné


In [83]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Retrieve data from the feature store
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)

# Get the data from the feature store
aq_past_df = air_quality_fg.read()

# Ensure the 'date' column is in datetime format
aq_past_df['date'] = pd.to_datetime(aq_past_df['date'], errors='coerce').dt.date  # Convert to date only

# Manually specify the dates for the past 3 days
past_dates = [today - datetime.timedelta(days=i) for i in range(1, 4)]

# Filter the DataFrame for rows that match the past 3 days
aq_past_df_filtered = aq_past_df[aq_past_df['date'].isin(past_dates)]

# Merge the past data with the new data
aq_today_df['pm25_lag_1'] = aq_past_df_filtered[aq_past_df_filtered['date'] == past_dates[0]]['pm25'].values
aq_today_df['pm25_lag_2'] = aq_past_df_filtered[aq_past_df_filtered['date'] == past_dates[1]]['pm25'].values
aq_today_df['pm25_lag_3'] = aq_past_df_filtered[aq_past_df_filtered['date'] == past_dates[2]]['pm25'].values

# Ensure the new features are of type float64 (double)
aq_today_df['pm25_lag_1'] = aq_today_df['pm25_lag_1'].astype('float64')
aq_today_df['pm25_lag_2'] = aq_today_df['pm25_lag_2'].astype('float64')
aq_today_df['pm25_lag_3'] = aq_today_df['pm25_lag_3'].astype('float64')

# Insert new data into the feature group
air_quality_fg.insert(aq_today_df)


aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.98s) 
2024-11-13 11:04:49,830 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163417/fs/1154120/fg/1345924


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163417/jobs/named/air_quality_1_offline_fg_materialization/executions
2024-11-13 11:04:59,038 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163417/fs/1154120/fg/1345924


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163417/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x294deab60>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669721
         }
       },
       "result": {
         "observed_value": 23.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T10:04:59.000038Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [76]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-12 12:53:29,432 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163417/fs/1154120/fg/1345925


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163417/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x292351ae0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669723
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T11:53:29.000432Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
